<a href="https://colab.research.google.com/github/cephee05/MachineLearningCertification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

# Charger les données d'entraînement
train_ds = tf.data.TextLineDataset("train-data.tsv")
test_ds = tf.data.TextLineDataset("valid-data.tsv")

# Retirer l'en-tête
train_lines = train_ds.skip(1)
test_lines = test_ds.skip(1)

# Fonction pour transformer chaque ligne en (texte, label)
def parse_line(line):
    parts = tf.strings.split(line, "\t")
    label = tf.where(parts[0] == "spam", 1, 0)
    text = parts[1]
    return text, label

train_data = train_lines.map(parse_line)
test_data = test_lines.map(parse_line)

# Mettre en batch et buffer
train_data = train_data.shuffle(1000).batch(32)
test_data = test_data.batch(32)

# Créer le vectorizer
vectorizer = TextVectorization(max_tokens=1000, output_mode='tf-idf')
vectorizer.adapt(train_data.map(lambda text, label: text))

# Créer le modèle
model = tf.keras.Sequential([
    vectorizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # proba entre 0 et 1
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=5)

# Fonction demandée
def predict_message(pred_text):
    prob = model.predict([pred_text])[0][0]
    label = "spam" if prob > 0.5 else "ham"
    return [float(prob), label]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Convertir en tenseur pour que TensorFlow accepte l'entrée
    input_text = tf.constant([pred_text])

    prob = model.predict(input_text)[0][0]
    label = "spam" if prob > 0.5 else "ham"
    return [float(prob), label]

# Tester avec un message
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
